In [16]:
import uproot
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import math
from scipy.fft import fft, fftfreq
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
from plotly.subplots import make_subplots
from plotly import tools
import plotly.offline as pyo
import sys

In [17]:
Runs = []
Times = []
Temp_df = {'AT':[],'AB':[],'BT':[],'BB':[],'mean':[]}
Run_txt = input("Enter the text file of run numbers: ")
with open(Run_txt) as f:
    for line in f:
        currentline = line.split(",")
        print(currentline)
        Runs.append(currentline[0])
        Times.append(int(currentline[1]))
        Temp_df['AT'].append(float(currentline[2]))
        Temp_df['AB'].append(float(currentline[3]))
        Temp_df['BT'].append(float(currentline[4]))
        Temp_df['BB'].append(float(currentline[5]))
        temp_mean = (float(currentline[2])+float(currentline[3])+float(currentline[4])+float(currentline[5]))/4
        Temp_df['mean'].append(temp_mean)
        
for i in range(len(Runs)):
    Runs[i] = int(Runs[i])
    Times[i] = datetime.fromtimestamp(int(Times[i])-3600)
print(Runs)

Enter the text file of run numbers: Runs.txt
['10772', '1707185580', '297.1', '295.55', '297.44', '295.69\n']
['10775', '1707234480', '298.54', '296.48', '298.88', '296.52\n']
['10841', '1707319440', '228.94', '209.0', '231.25', '208.32\n']
['10871', '1707364680', '179.69', '160.36', '181.34', '160.3\n']
['10874', '1707409380', '157.57', '142.83', '159.85', '141.19\n']
['10880', '1707420120', '139.94', '126.15', '141.89', '126.44\n']
['10893', '1707444000', '126.73', '113.93', '127.24', '113.98\n']
['10894', '1707530400', '128.47', '114.96', '128.7', '115.03\n']
['10922', '1707578220', '128.19', '114.39', '128.6', '114.38\n']
['10924', '1707591600', '128.67', '114.5', '128.8', '114.5\n']
['10925', '1707616800', '128.21', '113.64', '128.44', '114.24\n']
['10926', '1707663600', '126.54', '112.12', '126.83', '112.12\n']
['11015', '1707919200', '117.16', '99.01', '117.42', '99.01\n']
['11041', '1707958800', '120.98', '99.75', '121.07', '99.73\n']
['11042', '1708005600', '120.79', '98.43', 

In [18]:
Argon_df = pd.read_excel("SBND_LAr_level.xlsx")
print(Argon_df['Lar_level'][27])
Argon_df['Lar_level'] = Argon_df['Lar_level']/Argon_df['Lar_level'][28]*100
#print(datetime.fromtimestamp(Argon_df['Time'][0]))

516.42


In [19]:
wire_plane_list = ['UB','VB','YB','UA','VA','YA']
run_df = {'run':[],'run_median':[],'run_mean':[]}
for i in range(len(Runs)):
    Runs[i] = int(Runs[i])
    files =uproot.open(f"/Users/danielcarber/Documents/SBND/Noise Analysis/noise_output_run{Runs[i]}.root")
    files['tpc_noise;1'].keys()
    raw_rms = files['tpc_noise;1']['raw_rms'].array().to_list()
    Noise_df = {'Channel_id':[],'Raw_rms':[],'wire_plane':[]}
    for r,rms in enumerate(raw_rms):
        Noise_df['Channel_id'].append(r)
        if rms == 0:
            Noise_df['Raw_rms'].append(None)
        else:
            Noise_df['Raw_rms'].append(rms)
        if r <1984:
            Noise_df['wire_plane'].append('UB')
        elif r<3968:
            Noise_df['wire_plane'].append('VB')
        elif r<5632:
            Noise_df['wire_plane'].append('YB')
        elif r<7616:
            Noise_df['wire_plane'].append('UA')
        elif r<9600:
            Noise_df['wire_plane'].append('VA')
        else:
            Noise_df['wire_plane'].append('YA')
    Noise_df = pd.DataFrame(Noise_df)
    median = []
    mean = []
    for plane in wire_plane_list:
        mask = Noise_df['wire_plane'] == plane
        mask_1 = Noise_df['Raw_rms'] > -10
        median.append(np.median(Noise_df['Raw_rms'][mask][mask_1]))
        mean.append(np.mean(Noise_df['Raw_rms'][mask][mask_1]))
    median.append(np.median(Noise_df['Raw_rms'][mask_1]))
    mean.append(np.mean(Noise_df['Raw_rms'][mask_1]))
    run_df['run_median'].append(median)
    run_df['run_mean'].append(mean)
    run_df['run'].append(Runs[i])

In [20]:
def Extract(lst,pos):
    return [item[pos] for item in lst]
print(Times)

[datetime.datetime(2024, 2, 5, 19, 13), datetime.datetime(2024, 2, 6, 8, 48), datetime.datetime(2024, 2, 7, 8, 24), datetime.datetime(2024, 2, 7, 20, 58), datetime.datetime(2024, 2, 8, 9, 23), datetime.datetime(2024, 2, 8, 12, 22), datetime.datetime(2024, 2, 8, 19, 0), datetime.datetime(2024, 2, 9, 19, 0), datetime.datetime(2024, 2, 10, 8, 17), datetime.datetime(2024, 2, 10, 12, 0), datetime.datetime(2024, 2, 10, 19, 0), datetime.datetime(2024, 2, 11, 8, 0), datetime.datetime(2024, 2, 14, 7, 0), datetime.datetime(2024, 2, 14, 18, 0), datetime.datetime(2024, 2, 15, 7, 0), datetime.datetime(2024, 2, 15, 18, 0), datetime.datetime(2024, 2, 16, 7, 0), datetime.datetime(2024, 2, 16, 18, 0), datetime.datetime(2024, 2, 17, 6, 0), datetime.datetime(2024, 2, 17, 10, 59), datetime.datetime(2024, 2, 17, 18, 0), datetime.datetime(2024, 2, 18, 7, 0), datetime.datetime(2024, 2, 18, 11, 0), datetime.datetime(2024, 2, 18, 18, 0), datetime.datetime(2024, 2, 19, 13, 38), datetime.datetime(2024, 2, 19, 18

In [21]:
#filename = f"RMS_plots_planeB_{Run_num}.pdf"
dt = timedelta(hours=12)
fig = make_subplots(rows=3,cols=2,subplot_titles = (f'UB Median RMS vs Time','UA Median RMS vs Time',f'VB Median RMS vs Time','VA Median RMS vs Time',f'YB Median RMS vs Time','YA Median RMS vs Time',),
                                                   specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}], [{"secondary_y": True}, {"secondary_y": True}]])
#mask = Noise_df['wire_plane'] == 'UB'
#median = np.median(Noise_df['Raw_rms'][mask])
#mean = np.mean(Noise_df['Raw_rms'][mask])
#fig.add_trace(go.Histogram(x=Noise_df['Raw_rms'][mask],marker_color = 'red',xbins=dict(start = median - 5,end = median+5,size=.05)),row = 1, col =2)
'''
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],0),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 1, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],1),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 2, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],2),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 3, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],3),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 1, col = 2)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],4),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 2, col = 2)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],5),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 3, col = 2)
'''
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],0),marker_color = 'red',mode='lines+markers',name = 'RMS'),row = 1, col = 1,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],1),marker_color = 'red',mode='lines+markers',showlegend=False),row = 2, col = 1,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],2),marker_color = 'red',mode='lines+markers',showlegend=False),row = 3, col = 1,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],3),marker_color = 'red',mode='lines+markers',showlegend=False),row = 1, col = 2,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],4),marker_color = 'red',mode='lines+markers',showlegend=False),row = 2, col = 2,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],5),marker_color = 'red',mode='lines+markers',showlegend=False),row = 3, col = 2,secondary_y = False)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',name = 'Temp A Plane Top',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',name = 'Temp A Plane Bottom',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',name = 'Temp B. Plane Top',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',name = 'Temp B Plane Bottom',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)
def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    # fix indentation 
    positions = ['top center', 'bottom center']  # you can add more: left center ...
    return [positions[i % len(positions)] for i in range(len(x))]

fig.update_traces(textposition=improve_text_position(Extract(run_df['run_median'],0)),textfont_size=12)

fig.update_layout(yaxis = dict(range = [1,3.5]),xaxis = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis3 = dict(range = [1,3.5]),xaxis2 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis5 = dict(range = [1,3.5]),xaxis3 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis7 = dict(range = [1,3.5]),xaxis4 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis9 = dict(range = [1,3.5]),xaxis5 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis11 = dict(range = [1,3.5]),xaxis6 = dict(range = [min(Times)-dt,max(Times)+dt]))



fig.update_yaxes(title_text = "Median RMS [ADC]", secondary_y=False,row = 1, col = 1)
fig.update_yaxes(title_text = "Median RMS [ADC]", secondary_y=False,row = 2, col = 1)
fig.update_yaxes(title_text = "Median RMS [ADC]", secondary_y=False,row = 3, col = 1)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 1, col = 1)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 2, col = 1)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 3, col = 1)
fig.update_xaxes(title_text = "Time",row = 1, col = 1)
fig.update_xaxes(title_text = "Time",row = 2, col = 1)
fig.update_xaxes(title_text = "Time",row = 3, col = 1)
fig.update_xaxes(title_text = "Time",row = 1, col = 2)
fig.update_xaxes(title_text = "Time",row = 2, col = 2)
fig.update_xaxes(title_text = "Time",row = 3, col = 2)
fig.update_yaxes(title_text = "Median RMS [ADC]", secondary_y=False,row = 1, col = 2)
fig.update_yaxes(title_text = "Median RMS [ADC]", secondary_y=False,row = 2, col = 2)
fig.update_yaxes(title_text = "Median RMS [ADC]", secondary_y=False,row = 3, col = 2)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 1, col = 2)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 2, col = 2)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 3, col = 2)


fig.update_layout(height = 1200, width = 1600,showlegend = True)
first_run = run_df['run'][0]
last_run = run_df['run'][-1]
fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Median_RMS_vs_{first_run}_{last_run}.png')
#fig.write_image(directory+filename)
fig.show()

In [22]:
#filename = f"RMS_plots_planeB_{Run_num}.pdf"
dt = timedelta(hours=12)
fig = make_subplots(rows=3,cols=2,subplot_titles = (f'UB Mean RMS vs Time','UA Mean RMS vs Time',f'VB Mean RMS vs Time','VA Mean RMS vs Time',f'YB Mean RMS vs Time','YA Mean RMS vs Time',),
                                                   specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}], [{"secondary_y": True}, {"secondary_y": True}]])
#mask = Noise_df['wire_plane'] == 'UB'
#median = np.median(Noise_df['Raw_rms'][mask])
#mean = np.mean(Noise_df['Raw_rms'][mask])
#fig.add_trace(go.Histogram(x=Noise_df['Raw_rms'][mask],marker_color = 'red',xbins=dict(start = median - 5,end = median+5,size=.05)),row = 1, col =2)
'''
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],0),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 1, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],1),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 2, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],2),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 3, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],3),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 1, col = 2)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],4),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 2, col = 2)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],5),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 3, col = 2)
'''
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_mean'],0),marker_color = 'red',mode='lines+markers',name = 'RMS'),row = 1, col = 1,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_mean'],1),marker_color = 'red',mode='lines+markers',showlegend=False),row = 2, col = 1,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_mean'],2),marker_color = 'red',mode='lines+markers',showlegend=False),row = 3, col = 1,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_mean'],3),marker_color = 'red',mode='lines+markers',showlegend=False),row = 1, col = 2,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_mean'],4),marker_color = 'red',mode='lines+markers',showlegend=False),row = 2, col = 2,secondary_y = False)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_mean'],5),marker_color = 'red',mode='lines+markers',showlegend=False),row = 3, col = 2,secondary_y = False)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',name = 'Temp A Plane Top',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AT'],marker_color = 'blue',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',name = 'Temp A Plane Bottom',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['AB'],marker_color = 'cyan',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',name = 'Temp B. Plane Top',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BT'],marker_color = 'green',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',name = 'Temp B Plane Bottom',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 1, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 2, col = 2,secondary_y = True)
fig.add_trace(go.Scatter(x=Times,y = Temp_df['BB'],marker_color = 'lime',mode='lines+markers',showlegend=False,opacity=0.5),row = 3, col = 2,secondary_y = True)
def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    # fix indentation 
    positions = ['top center', 'bottom center']  # you can add more: left center ...
    return [positions[i % len(positions)] for i in range(len(x))]

fig.update_traces(textposition=improve_text_position(Extract(run_df['run_mean'],0)),textfont_size=12)

fig.update_layout(yaxis = dict(range = [1,3.5]),xaxis = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis3 = dict(range = [1,3.5]),xaxis2 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis5 = dict(range = [1,3.5]),xaxis3 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis7 = dict(range = [1,3.5]),xaxis4 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis9 = dict(range = [1,3.5]),xaxis5 = dict(range = [min(Times)-dt,max(Times)+dt]))
fig.update_layout(yaxis11 = dict(range = [1,3.5]),xaxis6 = dict(range = [min(Times)-dt,max(Times)+dt]))



fig.update_yaxes(title_text = "Mean RMS [ADC]", secondary_y=False,row = 1, col = 1)
fig.update_yaxes(title_text = "Mean RMS [ADC]", secondary_y=False,row = 2, col = 1)
fig.update_yaxes(title_text = "Mean RMS [ADC]", secondary_y=False,row = 3, col = 1)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 1, col = 1)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 2, col = 1)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 3, col = 1)
fig.update_xaxes(title_text = "Time",row = 1, col = 1)
fig.update_xaxes(title_text = "Time",row = 2, col = 1)
fig.update_xaxes(title_text = "Time",row = 3, col = 1)
fig.update_xaxes(title_text = "Time",row = 1, col = 2)
fig.update_xaxes(title_text = "Time",row = 2, col = 2)
fig.update_xaxes(title_text = "Time",row = 3, col = 2)
fig.update_yaxes(title_text = "Mean RMS [ADC]", secondary_y=False,row = 1, col = 2)
fig.update_yaxes(title_text = "Mean RMS [ADC]", secondary_y=False,row = 2, col = 2)
fig.update_yaxes(title_text = "Mean RMS [ADC]", secondary_y=False,row = 3, col = 2)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 1, col = 2)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 2, col = 2)
fig.update_yaxes(title_text = "Temp [K]", secondary_y=True,row = 3, col = 2)


fig.update_layout(height = 1200, width = 1600,showlegend = True)
first_run = run_df['run'][0]
last_run = run_df['run'][-1]
fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Mean_RMS_vs_{first_run}_{last_run}.png')
#fig.write_image(directory+filename)
fig.show()

In [57]:
#filename = f"RMS_plots_planeB_{Run_num}.pdf"
dt = timedelta(hours=12)
fig = make_subplots(rows=1,cols=1,subplot_titles = ('TPC Noise, LAr level, and Temperature Vs. Time',),
                                                   specs=[[{"secondary_y": True}]])
#mask = Noise_df['wire_plane'] == 'UB'
#median = np.median(Noise_df['Raw_rms'][mask])
#mean = np.mean(Noise_df['Raw_rms'][mask])
#fig.add_trace(go.Histogram(x=Noise_df['Raw_rms'][mask],marker_color = 'red',xbins=dict(start = median - 5,end = median+5,size=.05)),row = 1, col =2)
'''
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],0),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 1, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],1),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 2, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],2),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 3, col = 1)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],3),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 1, col = 2)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],4),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 2, col = 2)
fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_median'],5),marker_color = 'red',text = run_df['run'],mode='lines+markers+text'),row = 3, col = 2)
'''

fig.add_trace(go.Scatter(x=Times,y = Extract(run_df['run_mean'],6),marker_color = 'red',mode='lines+markers',showlegend=True,name = 'Noise'),row = 1, col = 1,secondary_y = False)

fig.add_trace(go.Scatter(x=Times,y = Temp_df['mean'],marker_color = 'blue',mode='lines+markers',name = 'Average Temp',opacity=0.5),row = 1, col = 1,secondary_y = True)
fig.add_trace(go.Scatter(x=Argon_df['Time'],y = Argon_df['Lar_level'],marker_color = 'green',mode='lines+markers',name = 'Argon Level',opacity=0.5),row = 1, col = 1,secondary_y = True)

def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    # fix indentation 
    positions = ['top center', 'bottom center']  # you can add more: left center ...
    return [positions[i % len(positions)] for i in range(len(x))]

fig.update_traces(textposition=improve_text_position(Extract(run_df['run_median'],0)),textfont_size=12)

fig.update_layout(yaxis = dict(range = [1,3.5]),xaxis = dict(range = [min(Times)-dt,max(Times)+dt]))



fig.update_yaxes(title_text = "Median RMS [ADC]", secondary_y=False,row = 1, col = 1,titlefont = dict(size=20),tickfont = dict(size=20))

fig.update_yaxes(title_text = "Temp [K] and LAr Level %", secondary_y=True,row = 1, col = 1,titlefont = dict(size=20),tickfont = dict(size=20))

fig.update_xaxes(title_text = "Time",row = 1, col = 1,titlefont = dict(size=20),tickfont = dict(size=20))


fig.update_annotations(font_size=30)


fig.update_layout(height = 600, width = 1200,showlegend = True,font = dict(size=20),legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.90,
    bgcolor="LightSteelBlue",
        bordercolor="Black",
))
first_run = run_df['run'][0]
last_run = run_df['run'][-1]
fig.write_image(f'/Users/danielcarber/Documents/SBND/Noise Analysis/Plots/Mean_RMS+temp_vs_{first_run}_{last_run}.png')
#fig.write_image(directory+filename)
fig.show()